In [1]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
from nltk import ngrams
from collections import Counter
from rapidfuzz import fuzz
import os
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False, nb_workers=10)
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TOKENIZERS_PARALLELISM']='false'
test = pd.read_csv("data/epir_test.csv")


df1 = pd.read_csv("data/epir_train/articles.csv")
df2 = pd.read_csv("data/epir_train/life_situations.csv")
df3 = pd.read_csv("data/epir_train/news.csv")
df4 = pd.read_csv("data/epir_train/services.csv")
df1_orig = df1.copy()
df2_orig = df2.copy()
df3_orig = df3.copy()
df4_orig = df4.copy()
df1['file'] = "articles"
df2['file'] = "life_situations"
df3['file'] = 'news'
df4['file'] = 'services'
#articles
df1 = df1.rename({'content':'body'}, axis=1)
#news
df2['title'] = df2['title_main'].fillna("") + " " + df2['title_sub'].fillna("")
df2 = df2.rename({'instruction':'body', 
            'URL':'url'}, axis=1)
df2 = df2[['Unnamed: 0', 'id', 'sys_lang', 'title','body', 'url','file']]
#news
df3 = df3.drop('short_description', axis=1)
#services
df4 = df4.drop("title", axis=1)
df4 = df4.rename({'full_title':'title'}, axis=1)
df4['body'] = df4['description'].fillna("")+" "+df4['result_description'].fillna("")
df4 = df4[['Unnamed: 0', 'id', 'sys_lang', 'title','body', 'url','file']]
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [2]:
# import sys
# !{sys.executable} -m pip install  pandarallel

In [3]:
df['text'] = df['title'].fillna("")+ " " + df['body'].fillna("")
df['text'] = df['text'].str.strip()
df['text'] = df['text'].str.lower()
df = df[df['text']!=''].copy()

In [4]:
df['text'] = df['text'].str.replace('ақпарат өзектендіру сатысында.','', regex=False)
df['text'] = df['text'].str.replace('information is being updated.','', regex=False)
df['text'] = df['text'].str.replace('информация находится на стадии актуализации. ','', regex=False)
df['text'] = df['text'].str.replace("Dear citizens of the Republic of Kazakhstan! In order to prevent the spread of coronavirus infection in the Republic of Kazakhstan, the provision of services on paper media is possible through online reservations in the service centers of the population for the period of quarantine.Public services can be obtained online, as well as some services are available in the mobile app EgovMbile, Telegram-bot eGovKzBot2 .0 and social networks Facebook and VK.".lower(), '', regex=False)
df['text'] = df['text'].str.replace("Уважаемые граждане Республики Казахстан! Для предотвращения распространения коронавирусной инфекции в Республике Казахстан оказание услуг на бумажных носителях возможно через онлайн бронирование в ЦОНах на период карантина.Государственные услуги можно получить онлайн, а также некоторые услуги и сервисы доступны в мобильном приложении EgovMobile, Telegram-боте eGovKzBot2.0 и социальных сетях Facebook и VK.".lower(), '', regex=False)
df['text'] = df['text'].str.replace("Құрметті Қазақстан Республикасының азаматтары! Қазақстан Республикасында коронавирустық инфекцияның таралуын болдырмау үшін, қағаз түрінде қызмет көрсету карантин уақытында ХҚКО-на онлайн брондау арқылы көрсетілетін болады.Мемлекеттік қызметтерді онлайн алуға болады, сонымен қатар кейбір қызметтер мен сервистер ЕgovМоbile мобильді қосымшасында, eGovKzBot2.0 Telegram-ботында және Facebook пен VK әлеуметтік жүйелерінде қолжетімді.".lower(), '', regex=False)

df['text'] = df['text'].str.strip()


In [5]:
# #очищаем от только p#.### без текста
df = df[~df['text'].str.contains('^p[\d\.]+$',regex=True)].copy()

In [6]:
df = df[df['sys_lang'].isin(['ru','kk','qq','en'])]

In [7]:
import fasttext
model = fasttext.load_model('lid.176.bin')

def detect_language(query):
    for lang, prob in zip(*model.predict(query, k=176)):
        lang = lang.split("__")[-1]
        if lang in {'ru','kk','en'}:
            return lang

        
# detect_language(query)

In [8]:
test['language'] = test['question'].apply(lambda x: detect_language(x))

In [9]:
test['language'].value_counts()

ru    57
en    39
kk    28
Name: language, dtype: int64

In [10]:
###TODO: add kazakh обработка
df = df[df['sys_lang'] != 'qq'].copy()

### Создание авто датасета вопросов

In [8]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.to(device);

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [13]:
n = 400

df_ru_sample = []
mask = (df['sys_lang']=='ru') & (df['text'].apply(lambda x: len(x.split()))<1000) & (df['text'].apply(lambda x: len(x.split()))>20)
df_ru_sample.append(df[mask & (df['file']=='news')].sample(n=n))
df_ru_sample.append(df[mask & (df['file']=='articles')].sample(n=n))
df_ru_sample.append(df[mask & (df['file']=='life_situations')].sample(n=n))
df_ru_sample.append(df[mask & (df['file']=='services')].sample(n=n))
df_ru_sample = pd.concat(df_ru_sample, ignore_index=True)
df_ru_sample = df_ru_sample.sample(frac=1.0)

In [20]:
questions = []
for line in tqdm(df_ru_sample['text'].values.tolist()):
    #Prefix <LM>
    lm_text="<SC1>"+line
    lm_text += 'Пользователь задал вопрос: <extra_id_0> ?'
    # lm_text='<SC1>Принялся Кутузов рассказывать свою историю . Началось с того, что он был в армии, служил в артиллерии.'
    input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
    outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=False)
    questions.append(tokenizer.decode(outputs[0][1:]))


  0%|          | 0/1600 [00:00<?, ?it/s]/home/maxat/.conda/envs/py3.9/lib/python3.9/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1600/1600 [09:41<00:00,  2.75it/s]


In [22]:
df_ru_sample['question'] = questions

In [26]:
df_ru_sample['question'] = df_ru_sample['question'].str.replace('<extra_id_0>','').str.replace('</s>','')
df_ru_sample['question'] = df_ru_sample['question'].str.strip()
df_ru_sample['question'] = df_ru_sample['question'].str.lower()

In [29]:
import pickle
with open('data/df_ru_sample.pkl','wb') as f:
    pickle.dump(df_ru_sample, f)


In [45]:
from pymystem3 import Mystem
from string import punctuation
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
russian_stopwords = set(stopwords.words("russian"))
ru_stem = Mystem() 
punctuation = set(punctuation)

def lemmatize_ru(sentence):
    tokens = ru_stem.lemmatize(sentence)
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens).strip()
    return text


[nltk_data] Downloading package stopwords to /home/maxat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:

df['text_stemmed'] = df['text'].copy()
df.loc[df['sys_lang']=='ru','text_stemmed'] = df.loc[df['sys_lang']=='ru','text_stemmed'].parallel_apply(lambda x: lemmatize_ru(x))

In [16]:
import pickle
with open("data/df_ru_sample.pkl",'rb') as f:
    df_ru_sample = pickle.load(f)

In [19]:
df_ru_sample = df_ru_sample.merge(df[['Unnamed: 0', 'text_stemmed']], how='left', on='Unnamed: 0')

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from fastbm25 import fastbm25

lang2df = {}
for lang, dfTemp in df.groupby("sys_lang"):
    lang2df[lang] = {}
    lang2df[lang]['df'] = dfTemp
    
for lang in ['ru']:
    df_lang = lang2df[lang]['df']
    corpus = df_lang['text_stemmed'].values.tolist()
    ids = df_lang['Unnamed: 0'].values.tolist()
    lang2df[lang]['ids'] = ids
    lang2df[lang]['vectorizer'] = TfidfVectorizer()
    lang2df[lang]['X'] = lang2df[lang]['vectorizer'].fit_transform(corpus)
    corpus = [sent.split() for sent in corpus]
    lang2df[lang]['bm25'] = fastbm25(corpus, PARAM_K1=1.5, PARAM_B = 0.75, EPSILON=0.25)
    

In [42]:
from transformers import AutoTokenizer, AutoModel
import torch
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-10)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
ru_bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

def get_ru_embs(sentences):
    #Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, max_length=512, truncation=True, return_tensors='pt')
    #Compute token embeddings
    with torch.no_grad():
        model_output = ru_bert(**encoded_input)
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings

In [75]:
from sklearn.metrics.pairwise import linear_kernel
data_ru_for_ranker = []
lang = 'ru'
K = 5
service_coef = 1.2
for row_ind in tqdm(df_ru_sample.index):
    row = df_ru_sample.loc[row_ind]
    query = row['question'].lower()
    query_orig = query
    query = lemmatize_ru(query)
    
    query_vec = lang2df[lang]['vectorizer'].transform([query])
    scores = linear_kernel(query_vec, lang2df[lang]['X'])[0]
    
    top_k_indices = np.argpartition(scores, -K)[-K:]
    top_k_indices = top_k_indices[np.argsort(scores[top_k_indices])][::-1]
    top_k_ids = [lang2df[lang]['ids'][ind] for ind in top_k_indices]
    dfTemp = df[df['Unnamed: 0'].isin(top_k_ids)].copy()
    dfTemp['target']=0
    dfTemp.loc[dfTemp['Unnamed: 0']==row['Unnamed: 0'],'target'] = 1
    dfTemp['query_id'] = row['Unnamed: 0']
    if dfTemp['target'].sum()==0:
        continue
        
    query_bert_vec = get_ru_embs([query_orig])
    bert_vecs = get_ru_embs(dfTemp['text'].values.tolist())
    bert_vecs = bert_vecs/torch.norm(bert_vecs, dim=1).reshape(-1,1)
    query_bert_vec = query_bert_vec/torch.norm(query_bert_vec, dim=1).reshape(-1,1)
    bert_scores = linear_kernel(query_bert_vec, bert_vecs)[0]
    dfTemp['bert_score'] = bert_scores
    dfTemp['tf_idfscore'] = scores[top_k_indices]

    topK_corpus = [sent.split() for sent in dfTemp['text_stemmed'].values.tolist()]
    dfTemp['bm25_score'] = [lang2df[lang]['bm25'].similarity_bm25(query.split(),sent) for sent in topK_corpus]
       
    dfTemp['WRatio'] = dfTemp['text'].apply(lambda x: fuzz.WRatio(query_orig,x))
    dfTemp['token_sort_ratio'] = dfTemp['text'].apply(lambda x: fuzz.token_sort_ratio(query_orig,x))
    dfTemp['ratio'] = dfTemp['text'].apply(lambda x: fuzz.ratio(query_orig,x))
    data_ru_for_ranker.append(dfTemp)

100%|██████████| 1600/1600 [05:31<00:00,  4.82it/s]


In [76]:
data_ru_for_ranker = pd.concat(data_ru_for_ranker, ignore_index=True)

In [128]:
data_ru_for_ranker
not_to_consider = ['Unnamed: 0','id','sys_lang', 'projects','title','body','url','text','text_stemmed','query_id', 'target', 'WRatio',
 'token_sort_ratio',
 'ratio', 'bert_score']

In [129]:
FEATURES = []
for col in data_ru_for_ranker.columns:
    if col not in not_to_consider:
        FEATURES.append(col)

In [130]:
FEATURES

['file', 'tf_idfscore', 'bm25_score']

In [131]:
data_ru_for_ranker

,Unnamed: 0,id,sys_lang,projects,title,body,url,file,text,text_stemmed,target,query_id,bert_score,tf_idfscore,bm25_score,WRatio,token_sort_ratio,ratio
0,48160,13813,ru,miid,Правила подачи заявления на разведку твердых п...,Разъяснения по основным положениям,https://www.gov.kz/memleket/entities/miid/pres...,articles,правила подачи заявления на разведку твердых п...,правило подача заявление разведка твердый поле...,0,1484,0.605829,0.695308,16.418629,65.172414,37.500000,37.500000
1,197871,326712,ru,atyrau-tabigat,Список заявителей на добычу общераспространенн...,Список заявителей на добычу общераспространенн...,https://www.gov.kz/memleket/entities/atyrau-ta...,news,список заявителей на добычу общераспространенн...,список заявитель добыча общераспространенный п...,0,1484,0.623579,0.630436,19.168002,66.122449,27.710843,31.325301
2,1329,3684,ru,NaN,Выдача заключения об отсутствии или малозначит...,Информация находится на стадии актуализации. П...,https://beta2.egov.kz/services/3684?lang=ru,services,выдача заключения об отсутствии или малозначит...,выдача заключение отсутствие малозначительност...,0,1484,0.313703,0.577325,20.783058,43.448276,5.375348,5.375348
3,1484,3405,ru,NaN,Регистрация договора залога права недропользов...,Информация находится на стадии актуализации. Ч...,https://beta2.egov.kz/services/3405?lang=ru,services,регистрация договора залога права недропользов...,регистрация договор залог право недропользован...,1,1484,0.562846,0.568091,20.443493,60.000000,6.674338,6.674338
4,1873,4306,ru,NaN,Утверждение проекта разработки месторождений п...,Информация находится на стадии актуализации. У...,https://beta2.egov.kz/services/4306?lang=ru,services,утверждение проекта разработки месторождений п...,утверждение проект разработка месторождение по...,0,1484,0.539182,0.543687,19.078382,68.275862,29.629630,27.513228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,61655,118110,ru,adilet-abai,Как правильно оформить снос здания?,29 апреля 2021 года в Республике Казахстан утв...,https://www.gov.kz/memleket/entities/adilet-ab...,articles,как правильно оформить снос здания? 29 апреля ...,правильно оформлять снос здание 29 апрель 2021...,0,1819,0.588140,0.298703,13.300297,57.000000,6.407323,6.712433
1101,150320,416149,ru,almaty-ugask,Третий этаж строящегося ЖК Pine Hill снесут по...,С прошлого года Управление судится с застройщи...,https://www.gov.kz/memleket/entities/almaty-ug...,news,третий этаж строящегося жк pine hill снесут по...,третий этаж строиться жк pine hill снести реше...,0,1819,0.588707,0.265575,22.964999,85.500000,18.562092,19.084967
1102,190122,514201,ru,zhambyl-taraz,Сказано – сделано: освещено 50 улиц,В целях улучшения ночного облика Тараза и обес...,https://www.gov.kz/memleket/entities/zhambyl-t...,news,сказано – сделано: освещено 50 улиц в целях ул...,сказать – сделать освещать 50 улица цель улу...,0,1819,0.468668,0.221824,16.158896,57.000000,8.942203,9.051254
1103,1788,4019,ru,NaN,Выдача решения на проведение комплекса работ п...,Информация находится на стадии актуализации. Г...,https://beta2.egov.kz/services/4019?lang=ru,services,выдача решения на проведение комплекса работ п...,выдача решение проведение комплекс работа пост...,0,1819,0.636476,0.182770,12.931723,85.500000,19.166667,18.611111


In [132]:
from sklearn.model_selection import train_test_split
unique_ids = data_ru_for_ranker['query_id'].unique()
train_ids, valid_ids = train_test_split(unique_ids, test_size=0.3, random_state=42)

In [133]:
train_idx = data_ru_for_ranker.loc[data_ru_for_ranker['query_id'].isin(train_ids)].index
valid_idx = data_ru_for_ranker.loc[data_ru_for_ranker['query_id'].isin(valid_ids)].index

In [134]:
data_ru_for_ranker['file'] = data_ru_for_ranker['file'].astype(str)

In [135]:
from catboost import CatBoostRanker, CatBoostClassifier, Pool
train = Pool(
        data = data_ru_for_ranker.loc[train_idx, FEATURES],
        label = data_ru_for_ranker.loc[train_idx, 'target'],
        group_id = data_ru_for_ranker.loc[train_idx, 'query_id'],
        cat_features=['file']
    )

valid = Pool(
        data = data_ru_for_ranker.loc[valid_idx, FEATURES],
        label = data_ru_for_ranker.loc[valid_idx, 'target'],
        group_id = data_ru_for_ranker.loc[valid_idx, 'query_id'],
        cat_features=['file']
    )

In [136]:
ranker = CatBoostRanker(iterations=20_000,
                      random_state=42,
                      learning_rate=0.01,
                      loss_function='PairLogitPairwise')
ranker.fit(train,
     verbose=100,
     early_stopping_rounds=250,
     eval_set=valid,
     use_best_model=True
     )

Pairwise scoring loss functions on CPU do not support one hot features. OneHotMaxSize set to 1
0:	learn: 0.6822092	test: 0.6834712	best: 0.6834712 (0)	total: 77.9ms	remaining: 25m 58s
100:	learn: 0.2574646	test: 0.3100628	best: 0.3100628 (100)	total: 4.15s	remaining: 13m 36s
200:	learn: 0.1686939	test: 0.2434432	best: 0.2434432 (200)	total: 8.28s	remaining: 13m 35s
300:	learn: 0.1291124	test: 0.2224243	best: 0.2224243 (300)	total: 12.4s	remaining: 13m 28s
400:	learn: 0.1063223	test: 0.2175900	best: 0.2172498 (386)	total: 16.4s	remaining: 13m 23s
500:	learn: 0.0902726	test: 0.2173163	best: 0.2164585 (468)	total: 20.6s	remaining: 13m 19s
600:	learn: 0.0780286	test: 0.2182570	best: 0.2164585 (468)	total: 24.6s	remaining: 13m 15s
700:	learn: 0.0684039	test: 0.2192463	best: 0.2164585 (468)	total: 28.7s	remaining: 13m 10s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.2164585142
bestIteration = 468

Shrink model to first 469 iterations.


In [137]:
from sklearn.metrics.pairwise import linear_kernel

test = pd.read_csv("data/epir_test.csv")
test['index'] = None
K = 3
service_coef = 1.2
all_results = []
for row_ind in tqdm(test.index):
    row = test.loc[row_ind]
    query = row['question'].lower()
    query_orig = query
    lang = detect_language(query)
    # apply_service_coef = False
    if lang=='ru':
        query = lemmatize_ru(query)
    else:
        continue
   
    query_vec = lang2df[lang]['vectorizer'].transform([query])
    scores = linear_kernel(query_vec, lang2df[lang]['X'])[0]
    
    top_k_indices = np.argpartition(scores, -K)[-K:]
    top_k_indices = top_k_indices[np.argsort(scores[top_k_indices])][::-1]
    top_k_ids = [lang2df[lang]['ids'][ind] for ind in top_k_indices]
    dfTemp = df.set_index("Unnamed: 0").loc[top_k_ids].copy()
    

    query_bert_vec = get_ru_embs([query_orig])
    bert_vecs = get_ru_embs(dfTemp['text'].values.tolist())
    bert_vecs = bert_vecs/torch.norm(bert_vecs, dim=1).reshape(-1,1)
    query_bert_vec = query_bert_vec/torch.norm(query_bert_vec, dim=1).reshape(-1,1)
    bert_scores = linear_kernel(query_bert_vec, bert_vecs)[0]
      
    query_bert_vec = get_ru_embs([query_orig])
    bert_vecs = get_ru_embs(dfTemp['text'].values.tolist())
    bert_vecs = bert_vecs/torch.norm(bert_vecs, dim=1).reshape(-1,1)
    query_bert_vec = query_bert_vec/torch.norm(query_bert_vec, dim=1).reshape(-1,1)
    bert_scores = linear_kernel(query_bert_vec, bert_vecs)[0]
    dfTemp['bert_score'] = bert_scores
    dfTemp['tf_idfscore'] = scores[top_k_indices]

    topK_corpus = [sent.split() for sent in dfTemp['text_stemmed'].values.tolist()]
    dfTemp['bm25_score'] = [lang2df[lang]['bm25'].similarity_bm25(query.split(),sent) for sent in topK_corpus]
       
    dfTemp['WRatio'] = dfTemp['text'].apply(lambda x: fuzz.WRatio(query_orig,x))
    dfTemp['token_sort_ratio'] = dfTemp['text'].apply(lambda x: fuzz.token_sort_ratio(query_orig,x))
    dfTemp['ratio'] = dfTemp['text'].apply(lambda x: fuzz.ratio(query_orig,x))
    dfTemp['ranker_score'] = ranker.predict(dfTemp[FEATURES])
    
    dfTemp = dfTemp.sort_values('ranker_score', ascending=False)
    results = dfTemp.index.tolist()
#     all_results.append(results)
#     # break
    test.loc[row_ind, 'index'] = int(results[0])
#     # print(row['question'])
# test['index'] = all_results

100%|██████████| 124/124 [00:43<00:00,  2.84it/s]


In [138]:
prev_sub = pd.read_csv('submissions/sub38.csv')

In [139]:
test = prev_sum.merge(test, how='left', on='id')

In [140]:
test['index'] = test[['index_x','index_y']].fillna(method='ffill', axis=1)['index_y']

In [141]:
test.set_index('id')[['index']].to_csv("submissions/sub42.csv")

In [21]:
import spacy
en_stopwords = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')
def lemmatize_en(sentence):
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc \
          if (token.lemma_ not in en_stopwords) and \
          (token.lemma_ not in punctuation)])

df.loc[df['sys_lang']=='en','text_stemmed'] = df.loc[df['sys_lang']=='en','text_stemmed'].parallel_apply(lambda x: lemmatize_en(x))



In [22]:
from kaznlp.morphology.analyzers import AnalyzerDD
import os
from nltk.tokenize import word_tokenize
nltk.download('punkt')


analyzer = AnalyzerDD()
analyzer.load_model(os.path.join('kaznlp', 'morphology', 'mdl'))

def lemmatize_kk(sentence):
    tokens = word_tokenize(sentence)
    return " ".join([analyzer.analyze(token)[1][0].split("_")[0] for token in tokens if token not in punctuation])

df.loc[df['sys_lang']=='kk','text_stemmed'] = df.loc[df['sys_lang']=='kk','text_stemmed'].parallel_apply(lambda x: lemmatize_kk(x))


[nltk_data] Downloading package punkt to /home/maxat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
df['text_stemmed'] = df['text_stemmed'].str.lower()
df['text_stemmed'] = df['text_stemmed'].str.strip()


In [24]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import importlib
# import fastbm25
# importlib.reload( fastbm25 )
# from fastbm25 import fastbm25

# lang2df = {}
# for lang, dfTemp in df.groupby("sys_lang"):
#     lang2df[lang] = {}
#     lang2df[lang]['df'] = dfTemp
    
# for lang in tqdm(lang2df):
#     df_lang = lang2df[lang]['df']
#     corpus = df_lang['text_stemmed'].values.tolist()
#     lang2df[lang]['corpus'] = [sent.split() for sent in corpus]
#     ids = df_lang['Unnamed: 0'].values.tolist()
#     lang2df[lang]['ids'] = ids
#     lang2df[lang]['vectorizer'] = fastbm25(lang2df[lang]['corpus'], PARAM_K1=1.5, PARAM_B = 0.75, EPSILON=0.25)
    
#     # lang2df[lang]['X'] = lang2df[lang]['vectorizer'].transform(corpus)

In [25]:
# from sklearn.metrics.pairwise import linear_kernel
# test = pd.read_csv("data/epir_test.csv")
# test['index'] = None
# K=3
# all_results = []
# for row_ind in tqdm(test.index):
#     row = test.loc[row_ind]
#     query = row['question'].lower()
#     lang = detect_language(query)
    
#     if lang=='ru':
#         query = lemmatize_ru(query)
#     elif lang=='en':
#         query = lemmatize_en(query)
#     elif lang=='kk':
#         query = lemmatize_kk(query)
#     query = query.split()
#     result = lang2df[lang]['vectorizer'].top_k_sentence(query,k=K)
#     top_k_ids = [lang2df[lang]['ids'][el[1]] for el in result]
#     scores = [el[2] for el in result]
#     dfTemp = df.set_index("Unnamed: 0").loc[top_k_ids].copy()
#     dfTemp['scores'] = scores
#     ##коррекция для services и life_situations
#     # dfTemp.loc[dfTemp['file'].isin(['services','life_situations']),'scores'] = 1.5*dfTemp.loc[dfTemp['file'].isin(['services','life_situations']),'scores']
#     dfTemp = dfTemp.sort_values('scores', ascending=False)
#     results = dfTemp.index.tolist()
#     all_results.append(results)
#     # test.loc[row_ind, 'index'] = results
#     # print(row['question'])
# test['index'] = all_results

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from fastbm25 import fastbm25

lang2df = {}
for lang, dfTemp in df.groupby("sys_lang"):
    lang2df[lang] = {}
    lang2df[lang]['df'] = dfTemp
    
for lang in tqdm(lang2df):
    df_lang = lang2df[lang]['df']
    corpus = df_lang['text_stemmed'].values.tolist()
    ids = df_lang['Unnamed: 0'].values.tolist()
    lang2df[lang]['ids'] = ids
    lang2df[lang]['vectorizer'] = TfidfVectorizer()
    lang2df[lang]['X'] = lang2df[lang]['vectorizer'].fit_transform(corpus)
    corpus = [sent.split() for sent in corpus]
    lang2df[lang]['bm25'] = fastbm25(corpus, PARAM_K1=1.5, PARAM_B = 0.75, EPSILON=0.25)
    

100%|██████████| 3/3 [01:29<00:00, 29.83s/it]


In [27]:
# from sklearn.metrics.pairwise import linear_kernel
# service_questions = {'en':['how','where'],
#                      'kk':['қалай','қайда'],
#                      'ru':['как','где']}


# test = pd.read_csv("data/epir_test.csv")
# test['index'] = None
# K = 10
# service_coef = 1.2
# all_results = []
# for row_ind in tqdm(test.index):
#     row = test.loc[row_ind]
#     query = row['question'].lower()
#     query_orig = query
#     lang = detect_language(query)
#     apply_service_coef = False
#     if lang=='ru':
#         query = lemmatize_ru(query)
#     elif lang=='en':
#         query = lemmatize_en(query)
#     elif lang=='kk':
#         query = lemmatize_kk(query)
#     query_vec = lang2df[lang]['vectorizer'].transform([query])
#     scores = linear_kernel(query_vec, lang2df[lang]['X'])[0]
    
#     top_k_indices = np.argpartition(scores, -K)[-K:]
#     top_k_indices = top_k_indices[np.argsort(scores[top_k_indices])][::-1]
#     top_k_ids = [lang2df[lang]['ids'][ind] for ind in top_k_indices]
#     dfTemp = df.set_index("Unnamed: 0").loc[top_k_ids].copy()
#     dfTemp['tf_idfscore'] = scores[top_k_indices]
    
#     topK_corpus = [sent.split() for sent in dfTemp['text_stemmed'].values.tolist()]
#     dfTemp['bm25_score'] = [lang2df[lang]['bm25'].similarity_bm25(query.split(),sent) for sent in topK_corpus]
#     ##коррекция для services и life_situations
    
#     dfTemp['WRatio'] = dfTemp['text'].apply(lambda x: fuzz.WRatio(query_orig,x))
#     dfTemp['token_sort_ratio'] = dfTemp['text'].apply(lambda x: fuzz.token_sort_ratio(query_orig,x))
#     dfTemp['ratio'] = dfTemp['text'].apply(lambda x: fuzz.ratio(query_orig,x))
    
#     score_columns = ['tf_idfscore','bm25_score']
#     # for col in score_columns:
#     #     dfTemp[col] = 1/(K-dfTemp[col].argsort())
#     dfTemp['bm25_score'] *= 0.4
#     dfTemp['total_score'] = dfTemp[score_columns].sum(axis=1)
#     # if any(question in query_orig for question in service_questions[lang]):
#     dfTemp.loc[dfTemp['file'].isin(['services','life_situations']),'total_score'] = service_coef*dfTemp.loc[dfTemp['file'].isin(['services','life_situations']),'total_score']
    
#     dfTemp = dfTemp.sort_values('total_score', ascending=False)
#     results = dfTemp.index.tolist()
#     all_results.append(results)
#     # if row['id']==122:
#     #     break
#     # break
#     # test.loc[row_ind, 'index'] = int(results[0])
#     # print(row['question'])
# test['index'] = all_results

In [169]:
from transformers import AutoTokenizer, AutoModel
import torch
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-10)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
ru_bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

def get_ru_embs(sentences):
    #Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')
    #Compute token embeddings
    with torch.no_grad():
        model_output = ru_bert(**encoded_input)
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings

In [164]:
def get_ru_embs2(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')
    #Compute token embeddings
    with torch.no_grad():
        model_output = ru_bert(**encoded_input)
    return model_output[0][:,-1,:]

In [180]:
from sentence_transformers import SentenceTransformer, util
bert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')
# bert_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased', device='cpu')

In [181]:
from transformers import AutoTokenizer, AutoModel
import torch
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-10)
    return sum_embeddings / sum_mask

tokenizer_kk = AutoTokenizer.from_pretrained("kz-transformers/kaz-roberta-conversational")
kk_bert = AutoModel.from_pretrained("kz-transformers/kaz-roberta-conversational")

def get_kk_embs(sentences):
    #Tokenize sentences
    encoded_input = tokenizer_kk(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')
    #Compute token embeddings
    with torch.no_grad():
        model_output = kk_bert(**encoded_input)
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings

Some weights of RobertaModel were not initialized from the model checkpoint at kz-transformers/kaz-roberta-conversational and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [182]:
from sklearn.metrics.pairwise import linear_kernel
service_questions = {'en':['how','where'],
                     'kk':['қалай','қайда'],
                     'ru':['как','где']}


test = pd.read_csv("data/epir_test.csv")
test['index'] = None
K = 3
service_coef = 1.2
all_results = []
for row_ind in tqdm(test.index):
    row = test.loc[row_ind]
    query = row['question'].lower()
    query_orig = query
    lang = detect_language(query)
    apply_service_coef = False
    if lang=='ru':
        query = lemmatize_ru(query)
    elif lang=='en':
        query = lemmatize_en(query)
    elif lang=='kk':
        query = lemmatize_kk(query)
    query_vec = lang2df[lang]['vectorizer'].transform([query])
    scores = linear_kernel(query_vec, lang2df[lang]['X'])[0]
    
    top_k_indices = np.argpartition(scores, -K)[-K:]
    top_k_indices = top_k_indices[np.argsort(scores[top_k_indices])][::-1]
    top_k_ids = [lang2df[lang]['ids'][ind] for ind in top_k_indices]
    dfTemp = df.set_index("Unnamed: 0").loc[top_k_ids].copy()
    if lang =='en':
        query_bert_vec = bert_model.encode([query_orig])
        bert_vecs = bert_model.encode(dfTemp['text'].values.tolist())
        bert_scores = linear_kernel(query_bert_vec, bert_vecs)[0]
        dfTemp['total_score'] = bert_scores
    elif lang=='kk':
        query_bert_vec = get_kk_embs([query_orig])
        bert_vecs = get_kk_embs(dfTemp['text'].values.tolist())
        bert_vecs = bert_vecs/torch.norm(bert_vecs, dim=1).reshape(-1,1)
        query_bert_vec = query_bert_vec/torch.norm(query_bert_vec, dim=1).reshape(-1,1)
        bert_scores = linear_kernel(query_bert_vec, bert_vecs)[0]
        dfTemp['total_score'] = bert_scores
    # elif lang=='ru':
    #     query_bert_vec = get_ru_embs([query_orig])
    #     bert_vecs = get_ru_embs(dfTemp['text'].values.tolist())
    #     bert_vecs = bert_vecs/torch.norm(bert_vecs, dim=1).reshape(-1,1)
    #     query_bert_vec = query_bert_vec/torch.norm(query_bert_vec, dim=1).reshape(-1,1)
    #     bert_scores = linear_kernel(query_bert_vec, bert_vecs)[0]
    #     dfTemp['total_score'] = bert_scores
    else:
        dfTemp['tf_idfscore'] = scores[top_k_indices]

        topK_corpus = [sent.split() for sent in dfTemp['text_stemmed'].values.tolist()]
        dfTemp['bm25_score'] = [lang2df[lang]['bm25'].similarity_bm25(query.split(),sent) for sent in topK_corpus]
        ##коррекция для services и life_situations

        dfTemp['WRatio'] = dfTemp['text'].apply(lambda x: fuzz.WRatio(query_orig,x))
        dfTemp['token_sort_ratio'] = dfTemp['text'].apply(lambda x: fuzz.token_sort_ratio(query_orig,x))
        dfTemp['ratio'] = dfTemp['text'].apply(lambda x: fuzz.ratio(query_orig,x))

        score_columns = ['tf_idfscore','bm25_score']
        # for col in score_columns:
        #     dfTemp[col] = 1/(K-dfTemp[col].argsort())
        dfTemp['bm25_score'] *= 0.4
        dfTemp['total_score'] = dfTemp[score_columns].sum(axis=1)
        # if any(question in query_orig for question in service_questions[lang]):
    dfTemp.loc[dfTemp['file'].isin(['services','life_situations']),'total_score'] = service_coef*dfTemp.loc[dfTemp['file'].isin(['services','life_situations']),'total_score']

    dfTemp = dfTemp.sort_values('total_score', ascending=False)
    results = dfTemp.index.tolist()
    all_results.append(results)
    # break
    # test.loc[row_ind, 'index'] = int(results[0])
    # print(row['question'])
test['index'] = all_results

100%|██████████| 124/124 [00:17<00:00,  6.99it/s]


In [183]:
test_vars = test.set_index('id')['index'].explode().reset_index()
test_vars = test_vars.rename({'level_0':'id'}, axis=1)
test_vars = test_vars.merge(test[['id','question']],
                how='left', on='id')
test_vars = test_vars.merge(df[['Unnamed: 0','text','file']].rename({'Unnamed: 0':'index'}, axis=1),
                       how='left', on='index')
test_vars['rank'] = test_vars.groupby('id').cumcount()+1
test_vars

,id,index,question,text,file,rank
0,1,2049,Алғашқы медициналық-санитариялық көмек көрсете...,алғашқы медициналық-санитариялық көмек көрсете...,services,1
1,1,179493,Алғашқы медициналық-санитариялық көмек көрсете...,сабақ 1. алғашқы медициналық көмек,news,2
2,1,139729,Алғашқы медициналық-санитариялық көмек көрсете...,денсаулық сақтау министрлігінің басшысы қостан...,news,3
3,2,647,How can I make an online reservation for servi...,issuance of id cards to stateless persons and ...,services,1
4,2,392,How can I make an online reservation for servi...,obtaining a quarantine certificate for movemen...,services,2
...,...,...,...,...,...,...
367,123,204849,Can I change my name online in Kazakhstan?,"how to change surname, name or patronymic chan...",life_situations,2
368,123,158059,Can I change my name online in Kazakhstan?,changes in legislation,news,3
369,124,128617,Documents needed for a name change in Kazakhstan.,"about changing the name, patronymic, surname t...",news,1
370,124,204850,Documents needed for a name change in Kazakhstan.,"how to change surname, name or patronymic what...",life_situations,2


In [184]:
test_vars[test_vars['rank'] ==1]

,id,index,question,text,file,rank
0,1,2049,Алғашқы медициналық-санитариялық көмек көрсете...,алғашқы медициналық-санитариялық көмек көрсете...,services,1
3,2,647,How can I make an online reservation for servi...,issuance of id cards to stateless persons and ...,services,1
6,3,155669,Online services Egov,new service for the pregnant available on egov...,news,1
9,4,156093,Telegram bot Egov servces,telegram bot features we are already used to t...,news,1
12,5,179713,Where can I find the latest updates and inform...,what public services can be obtained online wi...,news,1
15,6,116,How can I obtain a certificate from an antitub...,issuance of certificate from antituberculous o...,services,1
18,7,116,Process for obtaining an abstract from an anti...,issuance of certificate from antituberculous o...,services,1
21,8,116,Are there specific eligibility criteria or req...,issuance of certificate from antituberculous o...,services,1
24,9,1562,Разрешение на экспорт или импорт определенных ...,выдача разрешения на экспорт и (или) импорт от...,services,1
27,10,1562,Процесс получения разрешения на экспорт и/или ...,выдача разрешения на экспорт и (или) импорт от...,services,1


In [88]:
# df[(df['text'].str.contains('аккредита')) & (df['text'].str.contains('спорт')) & (df['sys_lang']=='ru')]

In [101]:
pd.set_option("display.max_rows", 125)

In [185]:
sub = test_vars[test_vars['rank'] ==1].copy()
sub['index'] = sub['index'].astype(int)
sub = sub.reset_index(drop=True)
sub = sub.set_index('id')
sub[['index']].to_csv("submissions/sub38.csv")

In [186]:
sub['index'] = sub['index'].astype(int)

In [187]:
prev_sub=pd.read_csv('submissions/sub37.csv')
prev_sub = prev_sub.set_index("id")
(prev_sub['index']==sub['index']).value_counts()


True     99
False    25
Name: index, dtype: int64

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.to(device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [13]:


df_ru_sample = []
mask = (df['sys_lang']=='ru') & (df['text'].apply(lambda x: len(x.split()))<1000) & (df['text'].apply(lambda x: len(x.split()))>20)
df_ru_sample.append(df[mask & (df['file']=='news')].sample(n=400))
df_ru_sample.append(df[mask & (df['file']=='articles')].sample(n=400))
df_ru_sample.append(df[mask & (df['file']=='life_situations')].sample(n=400))
df_ru_sample.append(df[mask & (df['file']=='services')].sample(n=400))
df_ru_sample = pd.concat(df_ru_sample, ignore_index=True)
df_ru_sample = df_ru_sample.sample(frac=1.0)

In [20]:
questions = []
for line in tqdm(df_ru_sample['text'].values.tolist()):
    #Prefix <LM>
    lm_text="<SC1>"+line
    lm_text += 'Пользователь задал вопрос: <extra_id_0> ?'
    # lm_text='<SC1>Принялся Кутузов рассказывать свою историю . Началось с того, что он был в армии, служил в артиллерии.'
    input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
    outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=False)
    questions.append(tokenizer.decode(outputs[0][1:]))


  0%|          | 0/1600 [00:00<?, ?it/s]/home/maxat/.conda/envs/py3.9/lib/python3.9/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1600/1600 [09:41<00:00,  2.75it/s]


In [22]:
df_ru_sample['question'] = questions

In [26]:
df_ru_sample['question'] = df_ru_sample['question'].str.replace('<extra_id_0>','').str.replace('</s>','')
df_ru_sample['question'] = df_ru_sample['question'].str.strip()
df_ru_sample['question'] = df_ru_sample['question'].str.lower()

In [29]:
import pickle
with open('data/df_ru_sample.pkl','wb') as f:
    pickle.dump(df_ru_sample, f)


In [239]:
df[df['sys_lang']=='ru'].iloc[-100]['text']

'возмещение затрат на повышение эффективности организации производства государственная поддержка субъектам индустриально-инновационной деятельности, направленная на повышение производительности труда, заключается в предоставлении мер государственной поддержки в рамках правил утвержденных приказом министра по инвестициям и развитию республики казахстан от 9 декабря 2015 года № 1194. подписание соглашения о возмещении затрат либо уведомление заявителя об отказе в предоставления меры государственной поддержки'

In [244]:
.apply('str')

0         бешенство – смертельная угроза для человека! б...
1         порядок администрирования налога на транспортн...
2         спорт миссия: формирование единой государствен...
5         полезные материалы об утверждении правил прием...
7         оперативная информация о чрезвычайных ситуация...
                                ...                        
207286                                                p7.06
207287                                                p7.05
207288                                                p3.03
207289                                               p3.041
207290                                               p21.09
Name: text, Length: 67669, dtype: object

,Unnamed: 0,id,sys_lang,projects,title,body,url,file,text
6391,9275,84298,ru,kyzylorda-kzo,Сообщение Кызылординской городской избиратель...,Постановлением Кызылординской городской избира...,https://www.gov.kz/memleket/entities/kyzylorda...,articles,сообщение кызылординской городской избиратель...
102992,205876,18,ru,NaN,Вы устраиваетесь на работу What kind of docume...,After the successful interview and the consent...,https://beta2.egov.kz/situations/18/96?lang=ru,life_situations,вы устраиваетесь на работу what kind of docume...
111782,110258,508139,ru,kyzylorda-kazaly,В Казалинске принято обращение о проблеме раст...,Сегодня на внеочередном заседании районного Об...,https://www.gov.kz/memleket/entities/kyzylorda...,news,в казалинске принято обращение о проблеме раст...
24347,27231,75889,ru,aqmola-zhaksy,Отчет о деятельности ГУ «Отдел занятости и соц...,ОТЧЕТ о деятельности ГУ «Отдел занятости и соц...,https://www.gov.kz/memleket/entities/aqmola-zh...,articles,отчет о деятельности гу «отдел занятости и соц...
196253,194729,606036,ru,ulytau,В области Ұлытау стартовала уборочная кампания,Из-за засухи в этом году объем урожая в област...,https://www.gov.kz/memleket/entities/ulytau/pr...,news,в области ұлытау стартовала уборочная кампания...
...,...,...,...,...,...,...,...,...,...
161247,159723,615726,ru,departament-kkbtu-akmolinsk,Сведения о количестве оказанных государственны...,Департаментом и территориальными управлениями ...,https://www.gov.kz/memleket/entities/departame...,news,сведения о количестве оказанных государственны...
50693,53577,107920,ru,pavlodar-pvl,Список победителей на занятие вакантных админи...,Список победителей на занятие вакантных админи...,https://www.gov.kz/memleket/entities/pavlodar-...,articles,список победителей на занятие вакантных админи...
174181,172657,567144,ru,maslihat-kargala,Личный прием граждан,Личный прием граждан Депутатом районного масли...,https://www.gov.kz/memleket/entities/maslihat-...,news,личный прием граждан личный прием граждан депу...
197377,195853,469525,ru,mfa-riga,Состоялась церемония инаугурации Президента Ре...,Во Дворце Независимости состоялась торжественн...,https://www.gov.kz/memleket/entities/mfa-riga/...,news,состоялась церемония инаугурации президента ре...
